In [21]:
import csv
import json
import os
from predictionguard import PredictionGuard

# Set your API key
api_key = os.getenv("PREDICTIONGUARD_API_KEY", "Oq62vYfSJRwjnFQcUnJy5PM3SRVejYtJCXWSxnfv")

# Initialize the PredictionGuard client
client = PredictionGuard(api_key=api_key)

# System content for the chatbot behavior based on your refined prompt
system_message = {
    "role": "system",
    "content": (
        "You are a language identification and translation system. Your task is to process a message and return it in the following way:\n"
        "1. If the message is entirely in English, return the message exactly as is.\n"
        "2. If the message is in both English and another language:\n"
        "    - If the foreign language part is a direct translation of the English text, return only the English.\n"
        "    - If the foreign language contains information that is not a direct translation, translate the entire message to English and return it.\n"
        "3. If the message is entirely in a foreign language, translate it into English and return it.\n"
        "4. If a message seems to be unrelated to reviews, is spam, or contains malicious content, output SPAM\n"
        "Do not provide any explanations or contextual information. Only return the final processed message."
    )
}

# Input and output CSV file paths
csv_file = 'test_file.csv'
output_file = 'translated_messages.csv'

# Read the CSV file with the messages and store rows
rows = []
with open(csv_file, newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Get the header and add a new column "translated messages"
    header = next(reader)
    header.append("translated messages")
    
    for row in reader:
        # Assuming the message is in the first column
        user_message = row[5]

        # Prepare the messages list for the chatbot
        messages = [
            system_message,
            {
                "role": "user",
                "content": user_message
            }
        ]

        # Send the message to the PredictionGuard API
        result = client.chat.completions.create(
            model="Hermes-3-Llama-3.1-8B",
            messages=messages
        )

        # Extract the chatbot's response
        response = result['choices'][0]['message']['content']

        # Append the translated message to the current row
        row.append(response)
        
        # Add the row to the list of rows
        rows.append(row)

# Write the updated data (with translated messages) back to a new CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(header)
    
    # Write the updated rows
    writer.writerows(rows)

print(f"Translated messages have been saved to {output_file}")


Translated messages have been saved to translated_messages.csv
